# Python Notebook to run requirement verfication for capella models

## Load model 

In [1]:
import capellambse
path_to_model = "./capella/moonbase/SGAC_Moon_Base/IP_New.aird"
model_capella = capellambse.MelodyModel(path_to_model)
model_capella

Cannot load PVMT extension: ValueError: Provided model does not have a PropertyValuePkg
Property values are not available in this model


## Get triples 

### Similarity analysis

In [14]:
from sentence_transformers import SentenceTransformer, util
import torch 
#req = "The settlement shall enable regular crew and cargo access to the lunar surface (and vice versa) via airlock or pressurized rover."
#req = "Airlock"
#req = "The Space Shuttle shall provide the basic transportation for all phases of the HST program including deployment, on-orbit servicing, and reboost or return to earth."
#req = "A scientific measurement capability is provided through a complement of up to four axial scientific instruments and a Fine Guidance Sensor."
#req = "All normal forward and return link data transmission shall be via the NASA Communications Network (NASCOM) and the Space Network (SN)."
#req = "The spacecraft shall have a mass < 11500kg."
#req = "One instrument shall be capable of detecting planets located more than 800 million kilometers (800 x 10^6 km) from their host star, in star systems situated at least 10 light-years away from the Solar System. Note: 1 light-year is approximately equal to 10 trillion kilometers (10 x 10^13 km)."
req = "Observational capability shall be provided by the instruments for the following: a. Visible photometric imaging b. Ultraviolet spectrophotometry c. Near infrared spectrophotometry (> 1 micron) and imaging"
req = "The settlement shall enable regular crew and cargo access to the lunar surface (and vice versa) via airlock or pressurized rover."
#req = "Crews shall be able to safely inhabit the settlement for up to 12 mo. at a time. NASA lunar orbital + surface reference mission (Antonsen and Taddeo, 2020)."
query = req

names = [comp.name for comp in model_capella.la.all_components]

model_emb = SentenceTransformer('all-mpnet-base-v2')

query_embedding = model_emb.encode(req, convert_to_tensor=True)

embeds = model_emb.encode(names, convert_to_tensor=True)

top_k = 10

cos_scores = util.cos_sim(query_embedding, embeds)[0]
top_results = torch.topk(cos_scores, k=top_k)


#print("\n\n======================\n\n")
print("======================")
print("Input query:", query)
print(f"\nTop most similar entries in Knowledge Graph:")

#iids = [iids[idx] for idx in top_results.indices.tolicst()]
#names= get_names([iids[idx] for idx in top_results.indices.tolist()], session) # inputs['artifact']['type'],
# scores = [top_results[idx] for idx in top_results.indices.tolist()]
# print(names)
for score, idx in zip(top_results[0], top_results[1]):
#for score, name in zip(top_results[0], names):
    # print(iids[idx], "(Score: {:.4f})".format(score))
    print(names[idx], "(Score: {:.4f})".format(score))




/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Input query: The settlement shall enable regular crew and cargo access to the lunar surface (and vice versa) via airlock or pressurized rover.

Top most similar entries in Knowledge Graph:
Settlement Peripherals & Payloads (Score: 0.5564)
Habitation (Score: 0.4687)
Pressurized Rover (Score: 0.3940)
Airlock (Score: 0.2821)
Lander (Score: 0.2003)
Surface Robotics (Score: 0.1974)
Uncrewed Surface Robotics (Score: 0.1813)
Structure, Shielding, & Storage (Score: 0.1680)
Thermal Control (Score: 0.1589)
Waste Management (Score: 0.1508)


### Reranking with Cross-Encoder

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "cross-encoder/ms-marco-MiniLM-L-12-v2"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

features = tokenizer(['How many people live in Berlin?', 'How many people live in Berlin?'], ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

tensor([[  9.4506],
        [-11.1605]])


In [3]:
from sentence_transformers import CrossEncoder
model_name = "cross-encoder/ms-marco-MiniLM-L-12-v2"
cross_encoder = CrossEncoder(model_name, max_length=512)

In [4]:
##### Re-Ranking #####
# Now, score all retrieved passages with the cross_encoder
query = req
entities = [names[idx] for idx in top_results.indices.tolist()]
cross_inp = [[query, entity] for entity in entities]
cross_scores = cross_encoder.predict(cross_inp)
hits = [{'entity': ent} for ent in entities]
# Sort results by the cross-encoder scores
for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]

# Output of top-5 hits from re-ranker
print("\n-------------------------\n")
print("Top-3 Cross-Encoder Re-ranker hits")
hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['cross-score'], hit["entity"]))


-------------------------

Top-3 Cross-Encoder Re-ranker hits
	-3.290	Settlement Peripherals & Payloads
	-5.324	Pressurized Rover
	-5.556	Airlock


### Reranking with LLM

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM#, # AutoModelForCausalLM, AutoTokenizer#, LlamaForCausalLM, LlamaTokenizer #, OPTForCausalLM, AutoModelForCausalLM
import torch 
MODEL_NAME ="openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5" #"mistralai/Mistral-7B-v0.1"
LOAD_8BIT = True 
# #LlamaForCausalLM.from_pretrained(MODEL_NAME,
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
        load_in_8bit=LOAD_8BIT,
        torch_dtype=torch.float16,
        device_map="auto",
        )
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if "open" in model.config._name_or_path:

    model.generation_config.pad_token_id = 0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.99s/it]
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `d

In [16]:
from transformers import GenerationConfig

def evaluate(
    prompt,
    model = None,
    tokenizer = None,
    stopping_criteria = None,
    device = 'cuda',
    **kwargs,
):

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        **kwargs,
    )
    
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            output_hidden_states= True,
            stopping_criteria= stopping_criteria
        )
        s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output, generation_output

MAX_NEW_TOKENS = 600
extra_args = {"max_new_tokens":MAX_NEW_TOKENS,
          #  "min_new_tokens": 256, 
             "do_sample":False,#True, 
            "num_beams" : 1, 
            "num_return_sequences" : 1, 
            #"no_repeat_ngram_size": 12, 
            "temperature": 0.5, 
            "top_p": 0.95,
          #  "begin_suppress_tokens": [2], 
            }

In [17]:
entities = [names[idx] for idx in top_results.indices.tolist()]
requirement = req
prompt = f"""GPT4 Correct User: Based on the requirement what is the most relevant entities? Return only a Python list object.
Entities are: {entities}
Requirement: {requirement}
<|end of turn|>GPT4 Correct Assistant: Entity:
"""
extra_args = {"max_new_tokens":MAX_NEW_TOKENS,
          #  "min_new_tokens": 256, 
             "do_sample":False,#True, 
            "num_beams" : 1, 
            "num_return_sequences" : 1, 
            #"no_repeat_ngram_size": 12, 
            "temperature": 0.5, 
            "top_p": 0.95,
          #  "begin_suppress_tokens": [2], 
            }
output, complete_output = evaluate(prompt, model = model, tokenizer= tokenizer,  device = "cuda", **extra_args) #

/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` i

In [18]:
print(prompt)

GPT4 Correct User: Based on the requirement what is the most relevant entities? Return only a Python list object.
Entities are: ['Settlement Peripherals & Payloads', 'Habitation', 'Pressurized Rover', 'Airlock', 'Lander', 'Surface Robotics', 'Uncrewed Surface Robotics', 'Structure, Shielding, & Storage', 'Thermal Control', 'Waste Management']
Requirement: The settlement shall enable regular crew and cargo access to the lunar surface (and vice versa) via airlock or pressurized rover.
<|end of turn|>GPT4 Correct Assistant: Entity:



In [19]:
print(output)

<s> GPT4 Correct User: Based on the requirement what is the most relevant entities? Return only a Python list object.
Entities are: ['Settlement Peripherals & Payloads', 'Habitation', 'Pressurized Rover', 'Airlock', 'Lander', 'Surface Robotics', 'Uncrewed Surface Robotics', 'Structure, Shielding, & Storage', 'Thermal Control', 'Waste Management']
Requirement: The settlement shall enable regular crew and cargo access to the lunar surface (and vice versa) via airlock or pressurized rover.
<|end of turn|>GPT4 Correct Assistant: Entity:
['Airlock', 'Pressurized Rover']
<|end of turn|><|end_of_turn|>


In [61]:
from ast import literal_eval

def parse_output(output, prompt_reason, tokenizer):

    output =output[len(tokenizer.decode(tokenizer.encode(prompt_reason))):]
    return output

def clean_text(text):
    text = text.split("<|end of turn|>")[0]
    text = text.strip()
    return text

parsed_output = parse_output(output, prompt, tokenizer)
parsed_output = clean_text(parsed_output)
parsed_output


def rerank(model, tokenizer, prompt, extra_args): 

    output, complete_output = evaluate(prompt, model = model, tokenizer= tokenizer,  device = "cuda", **extra_args)
    parsed_output = parse_output(output, prompt, tokenizer)
    parsed_output = clean_text(parsed_output)
    parsed_output = literal_eval(parsed_output)

    assert type(parsed_output) == list
    return parsed_output

entities = [names[idx] for idx in top_results.indices.tolist()]
requirement = req
prompt = f"""GPT4 Correct User: Based on the requirement what is the most relevant entities? Return only a Python list object.
Entities are: {entities}
Requirement: {requirement}
<|end of turn|>GPT4 Correct Assistant: Entity:
"""

entities = rerank(model, tokenizer, prompt, extra_args)



/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/paul/miniconda3

In [66]:
entity = "Habitation"
entity = model_capella.la.all_components.by_name(entity)

entity = "Exercise Facilities"
#model_capella.la.all_components.by_name(entity)

### Get Triples

In [4]:
#model_capella.la.all_components

In [22]:
from collections import deque

def get_neighbors_component_bfs(root_component, max_triples=None):
    queue = deque([root_component])
    triples = []
    visited = set()

    while queue:
        
        if max_triples and len(triples) >= max_triples:
            break  # Stop if the maximum number of triples has been reached
        #print(len(triples))
        component = queue.popleft()
        #print(component.name)
        ini_id = component.uuid

        if ini_id in visited:
            continue

        visited.add(ini_id)

        # Process direct component neighbors
        for comp in component.components:
            triples.append((comp.xtype, ini_id, comp.uuid, component.name, "directly_contains", comp.name))
            queue.append(comp)
        
        # Add parent to the queue
        try:
            triples.append((component.xtype, component.owner.uuid, ini_id, component.name, "is_contained_by", component.owner.name))
            queue.append(component.owner)
        except AttributeError: 
            print(component.name, "has no owner")

        # Add constraints
        for const in component.constraints:
            triples.append((const.xtype, ini_id, const.uuid, component.name, "constraint", const.name))

        # Add property values
        for prop in component.property_values:
            triples.append((prop.xtype, ini_id, prop.uuid, component.name, "has", prop.name +" " + str(prop.value)))

        try: 
            # Add component exchanges
            for exch in component.related_exchanges:
                if exch.target.owner.name != component.name: 
                    target = exch.target
                else: 
                    target = exch.source
                triples.append((target.owner.xtype, ini_id, target.owner.uuid, component.name, exch.name, target.owner.name))
                queue.append(target.owner)
                
        except AttributeError:
            pass

        try:    
            # Process functions and related exchanges
            for func in component.allocated_functions:
                triples.append((func.xtype, ini_id, func.uuid, component.name, "directly_performs", func.name))
                process_function_exchanges(func, triples, component, queue, visited)

        except AttributeError:
            pass

        #print(triples)
    
    #print(len(triples))
    print(visited)
    return triples

def process_function_exchanges(func, triples, component, queue, visited):
    # Process inputs and outputs of the function
    for input in func.inputs:
        if len(input.exchanges) == 1:
            source = input.exchanges[0]
            triples.append((source.xtype, func.uuid, source.source.owner.uuid, func.name, source.name, source.source.owner.name))
            add_parent_triple(source.source.owner, triples, queue, visited)

    for output in func.outputs:
        if len(output.exchanges) == 1:
            target = output.exchanges[0]
            triples.append((target.xtype, func.uuid, target.target.owner.uuid, func.name, target.name, target.target.owner.name))
            add_parent_triple(target.target.owner, triples, queue, visited)

def add_parent_triple(parent, triples, queue, visited):
    # Helper function to add a triple for the parent component and add it to the queue
    if parent is not None and parent.uuid not in visited:
        triples.append((parent.xtype, parent.uuid, parent.owner.uuid, parent.name, "is_performed_by", parent.owner.name))
        queue.append(parent.owner)


# Example usage:
# Assuming 'root_component' is your starting component
# and 'max_triples' is the maximum number of triples you want to insert
#results = get_neighbors_component_bfs(root_component, max_triples=50)
#root_component = model_capella.la.all_components.by_name(names[top_results[1][0]])

entities = ["Structure, Shielding, & Storage"]#,"Waste Management"]
results = []
for entity in entities:
    root_component = model_capella.la.all_components.by_name(entity)
    temp = get_neighbors_component_bfs(root_component, max_triples=25)

    for res in temp:
        results.append(res)

definition = ""
for triples in results: 
    definition += "|" + triples[3] +"|" +" " + "|" + triples[4] +"|" +" " +"|"+ triples[5] +"|" + "\n"

#print(definition) 

{'2f60ca8e-200e-420a-977a-5dda7c420641', '3585479a-6de3-49b0-ab88-787e38c40d36', '9e7257e0-56d7-4195-a9d0-79fbe540dd5c', '5c6a0f7b-5fd6-4b1c-8a58-b9693ba911cb'}


15
Wide Field Imagery sensor#1


NameError: name 'results' is not defined

In [24]:
#results

In [47]:
results[5:10]

[('org.polarsys.capella.core.data.la:LogicalFunction',
  '60fec714-d594-49ae-9a55-8608c0ebc77c',
  '2f60ca8e-200e-420a-977a-5dda7c420641',
  'Resupply Resources',
  'is_performed_by',
  'Lander'),
 ('org.polarsys.capella.core.data.la:LogicalComponent',
  '3585479a-6de3-49b0-ab88-787e38c40d36',
  '9e7257e0-56d7-4195-a9d0-79fbe540dd5c',
  'Logical System',
  'directly_contains',
  'Water Loop'),
 ('org.polarsys.capella.core.data.la:LogicalComponent',
  '3585479a-6de3-49b0-ab88-787e38c40d36',
  '14843f50-4b66-41ac-adad-b5df6db6cd21',
  'Logical System',
  'directly_contains',
  'Air Loop'),
 ('org.polarsys.capella.core.data.la:LogicalComponent',
  '3585479a-6de3-49b0-ab88-787e38c40d36',
  '9a8dc1a1-6edc-4804-ab8f-fbfd2534b8bd',
  'Logical System',
  'directly_contains',
  'Thermal Control'),
 ('org.polarsys.capella.core.data.la:LogicalComponent',
  '3585479a-6de3-49b0-ab88-787e38c40d36',
  'a33886c4-efe7-44e6-aaae-a37b6704118f',
  'Logical System',
  'directly_contains',
  'Food Loop')]

In [46]:
ids = [res[1] for res in results if res[0] == "org.polarsys.capella.core.data.la:LogicalComponent"]
set(ids)

{'1c807086-920c-401c-9165-a1874e465a8c',
 '3585479a-6de3-49b0-ab88-787e38c40d36',
 '69a2d84b-6907-45f4-8a13-11b83afa87e4'}

In [8]:
print(len(results))

70


#### What is this:

In [41]:
print("""|Habitation| |is_contained_by| |Logical System|\n|Habitation| |directly_performs| |Distribute Power|\n|Distribute Power| |Transmit Power| |Distribute Power|\n|Distribute Power| |is_performed_by| |Power|\n|Habitation| |directly_performs| |Provide Sleeping Quarters|\n|Provide Sleeping Quarters| |Move to Quarters| |Execute Work|\n|Execute Work| |is_performed_by| |Crew|\n|Provide Sleeping Quarters| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Provide Sleeping Quarters| |Go to bed| |Sleep|\n|Sleep| |is_performed_by| |Crew|\n|Habitation| |directly_performs| |Provide Exercise Facilities|\n|Provide Exercise Facilities| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Provide Exercise Facilities| |Move to Gym| |Exercise|\n|Exercise| |is_performed_by| |Crew|\n|Habitation| |directly_performs| |Provide Medical Services|\n|Provide Medical Services| |Move to Medical Station| |Perform Medical Care|\n|Perform Medical Care| |is_performed_by| |Crew|\n|Provide Medical Services| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Habitation| |directly_performs| |Kitchen|\n|Kitchen| |Cook Food| |Prepare Food|\n|Prepare Food| |is_performed_by| |Crew|\n|Kitchen| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Kitchen| |Bring food to Galley| |Consume Food|\n|Consume Food| |is_performed_by| |Crew|\n|Kitchen| |Generate Heat| |Control Temperature|\n|Control Temperature| |is_performed_by| |Thermal Control|\n|Crew| |is_contained_by| |Structure|\n|Crew| |directly_performs| |Execute Work|\n|Execute Work| |Exit Settlement| |Depressurize|\n|Depressurize| |is_performed_by| |Airlock|\n|Execute Work| |Crew Exits Rover| |Drive|\n|Drive| |is_performed_by| |Surface Robotics|\n|Execute Work| |Move to Workstation| |Make Decisions|\n|Make Decisions| |is_performed_by| |Crew|\n|Execute Work| |Plan Work| |Make Decisions|\n|Make Decisions| |is_performed_by| |Crew|\n|Execute Work| |Enter Settlement| |Re-Pressurize|\n|Re-Pressurize| |is_performed_by| |Airlock|\n|Execute Work| |Crew Returns to Rover| |Drive|\n|Drive| |is_performed_by| |Surface Robotics|\n|Execute Work| |Data Sent to Computer| |Receive Data|\n|Receive Data| |is_performed_by| |Communications|\n|Execute Work| |Move to Quarters| |Provide Sleeping Quarters|\n|Provide Sleeping Quarters| |is_performed_by| |Habitation|\n|Execute Work| |Generate Heat| |Control Temperature|\n|Control Temperature| |is_performed_by| |Thermal Control|\n|Crew| |directly_performs| |Prepare for EVA|\n|Prepare for EVA| |Enter Airlock| |Depressurize|\n|Depressurize| |is_performed_by| |Airlock|\n|Prepare for EVA| |Transfer to Rover| |Depressurize|\n|Depressurize| |is_performed_by| |Pressurized Rover|\n|Crew| |directly_performs| |Sleep|\n|Sleep| |Go to bed| |Provide Sleeping Quarters|\n|Provide Sleeping Quarters| |is_performed_by| |Habitation|\n|Crew| |directly_performs| |Prepare Food|\n|Prepare Food| |Harvest Food| |Produce Food|\n|Produce Food| |is_performed_by| |Food Loop|\n|Prepare Food| |Discard Packaging| |Compact Packaging Material|\n|Compact Packaging Material| |is_performed_by| |Waste Management|\n|Prepare Food| |Cook Food| |Kitchen|\n|Kitchen| |is_performed_by| |Habitation|\n|Crew| |directly_performs| |Hygiene (Shower, Teeth, etc.)|\n|Hygiene (Shower, Teeth, etc.)| |Clean up| |Provide Hygiene Facilities|\n|Provide Hygiene Facilities| |is_performed_by| |Water Loop|\n|Hygiene (Shower, Teeth, etc.)| |Collect Greywater| |Recover Water from Waste|\n|Recover Water from Waste| |is_performed_by| |Water Loop|\n|Hygiene (Shower, Teeth, etc.)| |Discard Packaging| |Compact Packaging Material|\n|Compact Packaging Material| |is_performed_by| |Waste Management|\n|Hygiene (Shower, Teeth, etc.)| |Collect Water Vapor| |Control Humidity|\n|Control Humidity| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Consume Oxygen|\n|Consume Oxygen| |Transport O2 to Crew| |Generate Oxygen|\n|Generate Oxygen| |is_performed_by| |Air Loop|\n|Consume Oxygen| |Metabolism| |Exhale CO2 and Water Vapor (incl. sweat)|\n|Exhale CO2 and Water Vapor (incl. sweat)| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Consume Water|\n|Consume Water| |Dispense Water| |Recover Water from Waste|\n|Recover Water from Waste| |is_performed_by| |Water Loop|\n|Consume Water| |Metabolism| |Urinate|\n|Urinate| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Consume Food|\n|Consume Food| |Bring food to Galley| |Kitchen|\n|Kitchen| |is_performed_by| |Habitation|\n|Consume Food| |Metabolism| |Excrete Feces and Produce Other Solid Waste (e.g. hair, nail clippings)|\n|Excrete Feces and Produce Other Solid Waste (e.g. hair, nail clippings)| |is_performed_by| |Crew|\n|Consume Food| |Brush Teeth| |Provide Hygiene Facilities|\n|Provide Hygiene Facilities| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Exercise|\n|Exercise| |Move to Gym| |Provide Exercise Facilities|\n|Provide Exercise Facilities| |is_performed_by| |Habitation|\n|Exercise| |Soil Clothing| |Remove Dirty Clothing|\n|Remove Dirty Clothing| |is_performed_by| |Crew|\n|Exercise| |To Shower| |Provide Hygiene Facilities|\n|Provide Hygiene Facilities| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Exhale CO2 and Water Vapor (incl. sweat)|\n|Exhale CO2 and Water Vapor (incl. sweat)| |Metabolism| |Consume Oxygen|\n|Consume Oxygen| |is_performed_by| |Crew|\n|Exhale CO2 and Water Vapor (incl. sweat)| |CO2 Collected| |Recover CO2 into O2|\n|Recover CO2 into O2| |is_performed_by| |Air Loop|\n|Exhale CO2 and Water Vapor (incl. sweat)| |Collect Water Vapor| |Control Humidity|\n|Control Humidity| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Urinate|\n|Urinate| |Metabolism| |Consume Water|\n|Consume Water| |is_performed_by| |Crew|\n|Urinate| |Collect Urine| |Recover Water from Waste|\n|Recover Water from Waste| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Excrete Feces and Produce Other Solid Waste (e.g. hair, nail clippings)|\n|Excrete Feces and Produce Other Solid Waste (e.g. hair, nail clippings)| |Metabolism| |Consume Food|\n|Consume Food| |is_performed_by| |Crew|\n|Excrete Feces and Produce Other Solid Waste (e.g. hair, nail clippings)| |Collect Waste| |Recover Nutrients from Solid Waste|\n|Recover Nutrients from Solid Waste| |is_performed_by| |Food Loop|\n|Crew| |directly_performs| |Make Decisions|\n|Make Decisions| |Data Relayed to Crew| |Transmit Data|\n|Transmit Data| |is_performed_by| |Communications|\n|Make Decisions| |Send Commands| |Receive Data|\n|Receive Data| |is_performed_by| |Communications|\n|Make Decisions| |Move to Workstation| |Execute Work|\n|Execute Work| |is_performed_by| |Crew|\n|Make Decisions| |Plan Work| |Execute Work|\n|Execute Work| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Evacuate to Another Module|\n|Evacuate to Another Module| |Escalate| |Split 68|\n|Split 68| |is_performed_by| |Crew|\n|Evacuate to Another Module| |Escalate| |Split 69|\n|Split 69| |is_performed_by| |Crew|\n|Evacuate to Another Module| |Escalate| |Split 70|\n|Split 70| |is_performed_by| |Crew|\n|Evacuate to Another Module| |Emergency Exit| |Isolate Module|\n|Isolate Module| |is_performed_by| |Emergency System|\n|Crew| |directly_performs| |Perform Medical Care|\n|Perform Medical Care| |Move to Medical Station| |Provide Medical Services|\n|Provide Medical Services| |is_performed_by| |Habitation|\n|Crew| |directly_performs| |Remove Dirty Clothing|\n|Remove Dirty Clothing| |Soil Clothing| |Exercise|\n|Exercise| |is_performed_by| |Crew|\n|Remove Dirty Clothing| |Put Clothes in Wash| |Wash Clothing|\n|Wash Clothing| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Put on Clean Clothing|\n|Put on Clean Clothing| |Take back clothing| |Wash Clothing|\n|Wash Clothing| |is_performed_by| |Water Loop|\n|Crew| |directly_performs| |Don gas masks|\n|Don gas masks| |Raise Toxic Spill Alarm| |Detect Toxic Spill|\n|Detect Toxic Spill| |is_performed_by| |Emergency System|\n|Don gas masks| |Raise Leak Alarm| |Detect Pressure Leak|\n|Detect Pressure Leak| |is_performed_by| |Emergency System|\n|Don gas masks| |Move to Spill| |Close & Isolate Module|\n|Close & Isolate Module| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Light Oxygen Candle|\n|Crew| |directly_performs| |Contain Spill or Leak|\n|Contain Spill or Leak| |Clean Up| |Close & Isolate Module|\n|Close & Isolate Module| |is_performed_by| |Crew|\n|Contain Spill or Leak| |Assess Situation| |Split 68|\n|Split 68| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Shelter-In-Place|\n|Shelter-In-Place| |Radiation Warning| |Solar Event Warning|\n|Solar Event Warning| |is_performed_by| |Emergency System|\n|Shelter-In-Place| |Assess Situation| |Split 69|\n|Split 69| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Close & Isolate Module|\n|Close & Isolate Module| |Move to Spill| |Don gas masks|\n|Don gas masks| |is_performed_by| |Crew|\n|Close & Isolate Module| |Raise Fire Alarm| |Detect Fire|\n|Detect Fire| |is_performed_by| |Emergency System|\n|Close & Isolate Module| |Clean Up| |Contain Spill or Leak|\n|Contain Spill or Leak| |is_performed_by| |Crew|\n|Close & Isolate Module| |Assess Situation| |Split 70|\n|Split 70| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Return to Normal|\n|Return to Normal| |Return to Normal| |Split 68|\n|Split 68| |is_performed_by| |Crew|\n|Return to Normal| |Return to Normal| |Split 69|\n|Split 69| |is_performed_by| |Crew|\n|Return to Normal| |Return to Normal| |Split 70|\n|Split 70| |is_performed_by| |Crew|\n|Return to Normal| |Reset Alarms| |Return to Normal|\n|Return to Normal| |is_performed_by| |Emergency System|\n|Crew| |directly_performs| |Split 68|\n|Split 68| |Assess Situation| |Contain Spill or Leak|\n|Contain Spill or Leak| |is_performed_by| |Crew|\n|Split 68| |Return to Normal| |Return to Normal|\n|Return to Normal| |is_performed_by| |Crew|\n|Split 68| |Escalate| |Evacuate to Another Module|\n|Evacuate to Another Module| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Split 69|\n|Split 69| |Assess Situation| |Shelter-In-Place|\n|Shelter-In-Place| |is_performed_by| |Crew|\n|Split 69| |Escalate| |Evacuate to Another Module|\n|Evacuate to Another Module| |is_performed_by| |Crew|\n|Split 69| |Return to Normal| |Return to Normal|\n|Return to Normal| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Split 70|\n|Split 70| |Assess Situation| |Close & Isolate Module|\n|Close & Isolate Module| |is_performed_by| |Crew|\n|Split 70| |Return to Normal| |Return to Normal|\n|Return to Normal| |is_performed_by| |Crew|\n|Split 70| |Escalate| |Evacuate to Another Module|\n|Evacuate to Another Module| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Set up Habitat|\n|Set up Habitat| |C&SU| |Re-Pressurize|\n|Re-Pressurize| |is_performed_by| |Airlock|\n|Set up Habitat| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Crew| |directly_performs| |Duplicate 75|\n|Duplicate 75| |Setup| |Set up Habitat|\n|Set up Habitat| |is_performed_by| |Crew|\n|Duplicate 75| |Setup| |Kitchen|\n|Kitchen| |is_performed_by| |Habitation|\n|Duplicate 75| |Setup| |Provide Exercise Facilities|\n|Provide Exercise Facilities| |is_performed_by| |Habitation|\n|Duplicate 75| |Setup| |Provide Sleeping Quarters|\n|Provide Sleeping Quarters| |is_performed_by| |Habitation|\n|Duplicate 75| |Setup| |Provide Medical Services|\n|Provide Medical Services| |is_performed_by| |Habitation|\n""")
#model_capella.la.all_functions.by_name("Duplicate 75").output

|Habitation| |is_contained_by| |Logical System|
|Habitation| |directly_performs| |Distribute Power|
|Distribute Power| |Transmit Power| |Distribute Power|
|Distribute Power| |is_performed_by| |Power|
|Habitation| |directly_performs| |Provide Sleeping Quarters|
|Provide Sleeping Quarters| |Move to Quarters| |Execute Work|
|Execute Work| |is_performed_by| |Crew|
|Provide Sleeping Quarters| |Setup| |Duplicate 75|
|Duplicate 75| |is_performed_by| |Crew|
|Provide Sleeping Quarters| |Go to bed| |Sleep|
|Sleep| |is_performed_by| |Crew|
|Habitation| |directly_performs| |Provide Exercise Facilities|
|Provide Exercise Facilities| |Setup| |Duplicate 75|
|Duplicate 75| |is_performed_by| |Crew|
|Provide Exercise Facilities| |Move to Gym| |Exercise|
|Exercise| |is_performed_by| |Crew|
|Habitation| |directly_performs| |Provide Medical Services|
|Provide Medical Services| |Move to Medical Station| |Perform Medical Care|
|Perform Medical Care| |is_performed_by| |Crew|
|Provide Medical Services| |Setup|

In [40]:
model_capella.la.all_components.by_name("Crew").allocated_functions[23].outputs[0].owner#.target

applied_property_value_groups,(Empty list)
applied_property_values,(Empty list)
available_in_states,(Empty list)
constraints,(Empty list)
description,
diagrams,(Empty list)
exchanges,(Empty list)
filtering_criteria,(Empty list)
functions,(Empty list)
inputs,"FunctionInputPort ""FIP 1"" (724a25e0-277d-4d3f-bfc5-686fc29e2685)"
is_leaf,True


#### print output

In [18]:
#print("\n")

print(definition)

|Structure, Shielding, & Storage| |is_contained_by| |Logical System|
|Structure, Shielding, & Storage| |directly_performs| |Protect Against Radiation & Debris|
|Structure, Shielding, & Storage| |directly_performs| |Support Weight of Internals + Shielding|
|Structure, Shielding, & Storage| |directly_performs| |Store Supplies & Equipment|
|Store Supplies & Equipment| |Move Resources from Lander to Habitat| |Resupply Resources|
|Resupply Resources| |is_performed_by| |Lander|
|Logical System| |directly_contains| |Water Loop|
|Logical System| |directly_contains| |Air Loop|
|Logical System| |directly_contains| |Thermal Control|
|Logical System| |directly_contains| |Food Loop|
|Logical System| |directly_contains| |Waste Management|
|Logical System| |directly_contains| |Command & Data Handling|
|Logical System| |directly_contains| |Communications|
|Logical System| |directly_contains| |Emergency System|
|Logical System| |directly_contains| |Power|
|Logical System| |directly_contains| |Habitatio

## LLM inference

### Load model 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM#, # AutoModelForCausalLM, AutoTokenizer#, LlamaForCausalLM, LlamaTokenizer #, OPTForCausalLM, AutoModelForCausalLM
import torch 
MODEL_NAME ="openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5" #"mistralai/Mistral-7B-v0.1"
LOAD_8BIT = True 
# #LlamaForCausalLM.from_pretrained(MODEL_NAME,
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
        load_in_8bit=LOAD_8BIT,
        torch_dtype=torch.float16,
        device_map="auto",
        )
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if "open" in model.config._name_or_path:

    model.generation_config.pad_token_id = 0

from transformers import StoppingCriteria, StoppingCriteriaList
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_token = input_ids[0][-1]
        for stop in self.stops:
            if tokenizer.decode(stop) == tokenizer.decode(last_token):
                return True
        return False
    
stop_words = ["Yes", "No"] #"\nYes"

stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]

#stop_words_ids = [5592, 5613, 1770]
#stop_words_ids = [torch.tensor(id) for id in stop_words_ids]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# template openchat : GPT4 Correct User: Hello<|end_of_turn|>GPT4 Correct Assistant: Hi<|end_of_turn|>GPT4 Correct User: How are you today?<|end_of_turn|>GPT4 Correct Assistant:
model.generation_config.pad_token_id = 0 # = GenerationConfig.from_pretrained("openchat/openchat_3.5")
test = "GPT4 Correct User: What is the colour of the sky?<|end_of_turn|>GPT4 Correct Assistant: "


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
MAX_NEW_TOKENS = 512 
extra_args = {"max_new_tokens":MAX_NEW_TOKENS,
          #  "min_new_tokens": 256, 
             "do_sample":True,#True, 
            "num_beams" : 1, 
            "num_return_sequences" : 1, 
            #"no_repeat_ngram_size": 12, 
            "temperature": 0.5, 
            "top_p": 0.95,
          #  "begin_suppress_tokens": [2], 
            }

In [9]:
from transformers import GenerationConfig
generation_config = GenerationConfig()
print(generation_config.do_sample)

generation_config = GenerationConfig(**extra_args)
print(generation_config.do_sample)

False
True


In [3]:
from transformers import GenerationConfig
generation_config = GenerationConfig()
print(generation_config.do_sample)
print(generation_config.no_repeat_ngram_size)
print(generation_config.temperature)
print(generation_config.top_p)
generation_config

In [2]:
from transformers import StoppingCriteria, StoppingCriteriaList
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_token = input_ids[0][-1]
        for stop in self.stops:
            if tokenizer.decode(stop) == tokenizer.decode(last_token):
                return True
        return False
    
stop_words = ["Yes", "No"] #"\nYes"

stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]

#stop_words_ids = [5592, 5613, 1770]

#stop_words_ids = [torch.tensor(id) for id in stop_words_ids]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [58]:
#tokenizer.vocab#[5592]
stop_words_ids

[tensor(5592), tensor([28705,    13,  5613]), tensor(1770)]

In [57]:
tokenizer.decode(stop_words_ids[1][2])

'Yes'

In [25]:
tokenizer.decode(stop_words_ids[0])

'Yes'

In [3]:
from transformers import GenerationConfig

def evaluate(
    prompt,
    model = None,
    tokenizer = None,
    stopping_criteria = None,
    device = 'cuda',
    **kwargs,
):

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        **kwargs,
    )
    
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            output_hidden_states= True,
            stopping_criteria= stopping_criteria,
            output_attentions=True,
        )
        s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output, generation_output

MAX_NEW_TOKENS = 600
extra_args = {"max_new_tokens":MAX_NEW_TOKENS,
          #  "min_new_tokens": 256, 
             "do_sample":False,#True, 
            "num_beams" : 1, 
            "num_return_sequences" : 1, 
            #"no_repeat_ngram_size": 12, 
            "temperature": 0.5, 
            "top_p": 0.95,
          #  "begin_suppress_tokens": [2], 
            }

In [8]:
# input = "What is a TT&C diplexer?"
# output = evaluate(input, model = model, tokenizer= tokenizer, device = "cuda", **extra_args) #

# print(output[0])

In [ ]:
input = "Hi"
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)


### reduced definition

In [ ]:
red = """|Spaceshuttle| |is_contained_by| |Structure|
|Spaceshuttle| |Transport Astronauts| |Astronauts|
|Astronauts| |is_contained_by| |Structure|
|Astronauts| |Transport Astronauts| |Spaceshuttle|
|Astronauts| |Services Telescope| |HUBBLE Space Telescope|
|Astronauts| |directly_performs| |Service Telescope|
|HUBBLE Space Telescope| |directly_contains| |Support System Module|
|HUBBLE Space Telescope| |directly_contains| |Instruments module|
|HUBBLE Space Telescope| |directly_contains| |Science&Instrument Control&Data Handling module|
|HUBBLE Space Telescope| |directly_contains| |Optical Telescope module|
|HUBBLE Space Telescope| |is_contained_by| |Structure|
|HUBBLE Space Telescope| |has| |Drymass in [KG] 11110.0|
|HUBBLE Space Telescope| |has| |Wet mass in [KG] 12218.0|
|HUBBLE Space Telescope| |Space network| |Tracking & Data Relay Satellite|
|HUBBLE Space Telescope| |Services Telescope| |Astronauts|
|HUBBLE Space Telescope| |Light| |Space|
|Tracking & Data Relay Satellite| |is_contained_by| |Structure|
|Tracking & Data Relay Satellite| |Space network| |HUBBLE Space Telescope|
|Tracking & Data Relay Satellite| |directly_performs| |Broadcast information from space|
|Broadcast information from space| |Information to earth| |Broadcast signal#1|
|Broadcast signal#1| |is_performed_by| |Antenna#1|
|Broadcast information from space| |Information to earth| |Broadcast signal#2|
|Broadcast signal#2| |is_performed_by| |Antenna#2|
|Broadcast information from space| |Information from earth| |Broadcast signal#1|
|Broadcast signal#1| |is_performed_by| |Antenna#1|
|Broadcast information from space| |Information from earth| |Broadcast signal#2|
|Broadcast signal#2| |is_performed_by| |Antenna#2|
|Space| |is_contained_by| |Structure|
|Space| |Light| |HUBBLE Space Telescope|
|Space| |directly_performs| |Diffuse light|
|Diffuse light| |Space light| |Collect light|
|Collect light| |is_performed_by| |Light collector|
|Diffuse light| |Space light| |Find and maintain a lock on guide stars|
|Find and maintain a lock on guide stars| |is_performed_by| |Locking sensor|
|Diffuse light| |Space light| |Collect sunlight energy#1|
|Collect sunlight energy#1| |is_performed_by| |Solar Array#1|
|Diffuse light| |Space light| |Collect sunlight energy#2|
|Collect sunlight energy#2| |is_performed_by| |Solar Array#2|"""

In [ ]:
orig = """|Spaceshuttle| |directly_contains| |Deployment Boom |
|Spaceshuttle| |is_contained_by| |Structure|
|Spaceshuttle| |Transport Astronauts| |Astronauts|
|Deployment Boom | |is_contained_by| |Spaceshuttle|
|Deployment Boom | |Deployment| |HUBBLE Space Telescope|
|Deployment Boom | |Return| |HUBBLE Space Telescope|
|Deployment Boom | |directly_performs| |Deploy Telescope|
|Deployment Boom | |directly_performs| |Catch satellite|
|Structure| |directly_contains| |HUBBLE Space Telescope|
|Structure| |directly_contains| |Tracking & Data Relay Satellite|
|Structure| |directly_contains| |Space|
|Structure| |directly_contains| |Astronauts|
|Structure| |directly_contains| |Spaceshuttle|
|Astronauts| |is_contained_by| |Structure|
|Astronauts| |Transport Astronauts| |Spaceshuttle|
|Astronauts| |Services Telescope| |HUBBLE Space Telescope|
|Astronauts| |directly_performs| |Service Telescope|
|HUBBLE Space Telescope| |directly_contains| |Support System Module|
|HUBBLE Space Telescope| |directly_contains| |Instruments module|
|HUBBLE Space Telescope| |directly_contains| |Science&Intrument Control&Data Handling module|
|HUBBLE Space Telescope| |directly_contains| |Optical Telescope module|
|HUBBLE Space Telescope| |is_contained_by| |Structure|
|HUBBLE Space Telescope| |has| |Drymass in [KG] 11110.0|
|HUBBLE Space Telescope| |has| |Wet mass in [KG] 12218.0|
|HUBBLE Space Telescope| |Space network| |Tracking & Data Relay Satellite|
|HUBBLE Space Telescope| |Deployment| |Deployment Boom |
|HUBBLE Space Telescope| |Return| |Deployment Boom |
|HUBBLE Space Telescope| |Services Telescope| |Astronauts|
|HUBBLE Space Telescope| |Light| |Space|
|Tracking & Data Relay Satellite| |is_contained_by| |Structure|
|Tracking & Data Relay Satellite| |Space network| |HUBBLE Space Telescope|
|Tracking & Data Relay Satellite| |directly_performs| |Broadcast information from space|
|Broadcast information from space| |Information to earth| |Broadcast signal#1|
|Broadcast signal#1| |is_performed_by| |Antenna#1|
|Broadcast information from space| |Information to earth| |Broadcast signal#2|
|Broadcast signal#2| |is_performed_by| |Antenna#2|
|Broadcast information from space| |Information from earth| |Broadcast signal#1|
|Broadcast signal#1| |is_performed_by| |Antenna#1|
|Broadcast information from space| |Information from earth| |Broadcast signal#2|
|Broadcast signal#2| |is_performed_by| |Antenna#2|
|Space| |is_contained_by| |Structure|
|Space| |Light| |HUBBLE Space Telescope|
|Space| |directly_performs| |Diffuse light|
|Diffuse light| |Space light| |Collect light|
|Collect light| |is_performed_by| |Light collector|
|Diffuse light| |Space light| |Find and maintain a lock on guide stars|
|Find and maintain a lock on guide stars| |is_performed_by| |Locking sensor|
|Diffuse light| |Space light| |Collect sunlight energy#1|
|Collect sunlight energy#1| |is_performed_by| |Solar Array#1|
|Diffuse light| |Space light| |Collect sunlight energy#2|
|Collect sunlight energy#2| |is_performed_by| |Solar Array#2|
"""

### excecute prompt

In [4]:
from transformers import GenerationConfig

#prompt_reason = """Based on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No" . Use Python scripting if you need to calculate something by starting with '''python... Let's think step-by-step. """
prompt_reason = """Based on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No". Use Python scripting if you need to calculate something by starting with '''python... Let's think step-by-step. """

# prompt_reason = """Based on the system description, is this requirement satisfied?
# 1. Read the information (entities and relations) given for the System carefully and identify the triples that are potentially useful in your analysis. 
# 2. Next try to establish if that information either implicitly or explicitly fulfills the requirement. 
# 3. State if the requirement is satisfied or not by stating Yes or No.

# Answer (Yes or No ONLY): """
## This could involve performing a gap analysis between the engineering system and the requirement. 
system_prompt ="GPT4 Correct Assistant: "# "ASSISTANT:"#"GPT4 Correct Assistant: "
user_prompt = "GPT4 Correct User: "#"HUMAN:"#"GPT4 Correct User: "
 

# """|Spaceshuttle| |is_contained_by| |Structure|
# |Spaceshuttle| |Transport Astronauts| |Astronauts|
# |Astronauts| |is_contained_by| |Structure|
# |Astronauts| |Transport Astronauts| |Spaceshuttle|
# |Astronauts| |Services Telescope| |HUBBLE Space Telescope|
# |Astronauts| |directly_performs| |Service Telescope|"""

definition = """
|Astronauts| |Services Telescope| |HUBBLE Space Telescope|
|Astronauts| |directly_performs| |Service Telescope|
|HUBBLE Space Telescope| |directly_contains| |Support System Module|
|HUBBLE Space Telescope| |directly_contains| |Instruments module|
|HUBBLE Space Telescope| |directly_contains| |Science&Instrument Control&Data Handling module|
|HUBBLE Space Telescope| |directly_contains| |Optical Telescope module|
|HUBBLE Space Telescope| |is_contained_by| |Structure|
|HUBBLE Space Telescope| |has| |Drymass in [KG] 11110.0|
|HUBBLE Space Telescope| |has| |Wet mass in [KG] 12218.0|
|HUBBLE Space Telescope| |Space network| |Tracking & Data Relay Satellite|
|HUBBLE Space Telescope| |Services Telescope| |Astronauts|
|HUBBLE Space Telescope| |Light| |Space|
|Tracking & Data Relay Satellite| |is_contained_by| |Structure|
|Tracking & Data Relay Satellite| |Space network| |HUBBLE Space Telescope|
|Tracking & Data Relay Satellite| |directly_performs| |Broadcast information from space|
|Broadcast information from space| |Information to earth| |Broadcast signal#1|
|Broadcast signal#1| |is_performed_by| |Antenna#1|
|Broadcast information from space| |Information to earth| |Broadcast signal#2|
|Broadcast signal#2| |is_performed_by| |Antenna#2|
|Broadcast information from space| |Information from earth| |Broadcast signal#1|
|Broadcast signal#1| |is_performed_by| |Antenna#1|
|Broadcast information from space| |Information from earth| |Broadcast signal#2|
|Broadcast signal#2| |is_performed_by| |Antenna#2|
|Space| |is_contained_by| |Structure|
|Space| |Light| |HUBBLE Space Telescope|
|Space| |directly_performs| |Diffuse light|
|Diffuse light| |Space light| |Collect light|
|Collect light| |is_performed_by| |Light collector|
|Diffuse light| |Space light| |Find and maintain a lock on guide stars|
|Find and maintain a lock on guide stars| |is_performed_by| |Locking sensor|
|Diffuse light| |Space light| |Collect sunlight energy#1|
|Collect sunlight energy#1| |is_performed_by| |Solar Array#1|
|Diffuse light| |Space light| |Collect sunlight energy#2|
|Collect sunlight energy#2| |is_performed_by| |Solar Array#2|
"""
definition = """|JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|
|JWST| |orbits| |Second LaGrange Point (L2)|
|JWST| |maintains_ZExcursion| "≤500,000 Km"
|JWST| |maintains_YExcursion| "≤800,000 Km"
|JWST| |avoids| |Eclipse|
|JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|
|JWST| |has_MissionLifetime| "5 years"
|JWST| |completes_Commissioning| within "6 months"
|JWST System| |delivers_RealTimeTelemetry| "92.5%"
|JWST System| |archives_RecordedData| "97%"
|JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|"""

req = "The Space Shuttle shall provide the basic transportation for all phases of the HST program including deployment, on-orbit servicing, and reboost or return to earth."
req = "The planned commissioning phase shall end no later than six months after launch."
prompt_reason = f"{user_prompt}System: {definition}\nRequirement: {req}\n{prompt_reason}\n"#{system_prompt}"# +"Let's analyze the given System and requirement step-by-step:"

#prompt_reason = "What is the colour of the sky?"
#final answer: yes
#prompt_reason = "Please answer the following question. Are there 50 states in the US. Answer (Yes or No ONLY):"
#output, complete_output = evaluate(prompt_reason, model = model, tokenizer= tokenizer, stopping_criteria = stopping_criteria, device = "cuda", **extra_args) #
output, complete_output = evaluate(prompt_reason, model = model, tokenizer= tokenizer, stopping_criteria = None, device = "cuda", **extra_args) #

/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` i

In [26]:
#model.config

In [21]:
#complete_output.sequences[0]

In [12]:
output = (0,{"scores": torch.zeros(8196).long()})

len(output[1]['scores'])

8196

In [28]:
print(output)

<s> GPT4 Correct User: System: |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|
|JWST| |orbits| |Second LaGrange Point (L2)|
|JWST| |maintains_ZExcursion| "≤500,000 Km"
|JWST| |maintains_YExcursion| "≤800,000 Km"
|JWST| |avoids| |Eclipse|
|JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|
|JWST| |has_MissionLifetime| "5 years"
|JWST| |completes_Commissioning| within "6 months"
|JWST System| |delivers_RealTimeTelemetry| "92.5%"
|JWST System| |archives_RecordedData| "97%"
|JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|
Requirement: The planned commissioning phase shall end no later than six months after launch.
Based on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No". Use Python scripting if you need to calculate something by starting with '''python... Let's think step-by-step. 

1. The commissioning phase of the JWST system is mentioned to be completed within six months

In [7]:
len(complete_output.scores)#.shape

75

In [27]:
#complete_output.scores#.loss

In [29]:
scores= torch.softmax(complete_output.scores,1)

TypeError: softmax() received an invalid combination of arguments - got (tuple, int), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)


In [2]:
import numpy as np 

#torch.tensor(complete_output.scores)
#torch.softmax(complete_output.scores[-1],1)

In [15]:
# complete_output.sequences

In [58]:
def calculate_sequence_uncertainty(output):

    scores= []

    ## calculate the geometric mean of the sequence (becoming an arithmetic mean log-probability) of the most likely token, ignoring that on generation another token can be chosen than the most likeliest. 
    for i in range(len(output.scores)):
        
        ## https://huggingface.co/docs/transformers/v4.36.1/en/internal/generation_utils#transformers.generation.SampleEncoderDecoderOutput
        ## gets probabilities from raw prediction scores of the lm head for each token
        score = torch.softmax(output.scores[i],1)
        ## gets the most likely token score
        score = torch.max(score).item()

        scores.append(score)

    ## https://huggingface.co/docs/transformers/perplexity
    log_scores = np.log(np.array(scores))#.shape
    mean = -np.sum(log_scores)/len(scores)
    return round(mean,2)

calculate_sequence_uncertainty(complete_output)

0.27

In [6]:
def get_attention_scores(output):
    # Specify the layers and heads you are interested in
    layers = [-1, -8, -16, -24]
    #heads = [0, 1, 2, 3, 4, ,5] ### create list for 32 heads
    heads = [num for num in range(32)]
    #heads =  # Example heads, modify as needed

    output_attention_scores = {}
    for layer in layers:
        for head in heads:
            # Get attention score of last generated token [-1] in specific layer [layer] sequence[0] and head [head]
            # AND Move to CPU and convert to float32 or float64
            
            attention_tensor = output.attentions[-1][layer][0][head][-1].to(device='cpu', dtype=torch.float32)

            # Convert to a list
            tensor_list = attention_tensor.tolist()

            # Store in the dictionary
            output_attention_scores[f"layer_{layer}_head_{head}_last_token"] = tensor_list

    return output_attention_scores

In [60]:
MODEL_NAME ="likenneth/honest_llama2_chat_7B"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5" #"mistralai/Mistral-7B-v0.1"
LOAD_8BIT = True 
# #LlamaForCausalLM.from_pretrained(MODEL_NAME,
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
        load_in_8bit=LOAD_8BIT,
        torch_dtype=torch.float16,
        device_map="auto",
        )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
MODEL_NAME ="likenneth/honest_llama2_chat_7B"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5" #"mistralai/Mistral-7B-v0.1"
LOAD_8BIT = True 
# #LlamaForCausalLM.from_pretrained(MODEL_NAME,
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
        load_in_8bit=LOAD_8BIT,
        torch_dtype=torch.float16,
        device_map="auto",
        )

In [ ]:
import sys 
sys.path.insert(0, "/home/jovyan/work/honest_llama2_chat_7B")

In [69]:
#model.model.layers[0].mlp
model.model.layers[0].self_attn#.head_out

LlamaAttention(
  (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
  (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
)

In [3]:
from baukit import Trace, TraceDict

def get_llama_activations_bau(model, prompt, device): 

    #HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(model.config.num_hidden_layers)]
    ## https://github.com/likenneth/honest_llama/issues/7
    HEADS = [f"model.layers.{i}.self_attn.o_proj" for i in range(model.config.num_hidden_layers)]
    MLPS = [f"model.layers.{i}.mlp" for i in range(model.config.num_hidden_layers)]
    #print(HEADS)
    with torch.no_grad():
        prompt = prompt.to(device)
        with TraceDict(model, HEADS+MLPS, retain_input=True) as ret:
           #output = model.generate(prompt, return_dict_in_generate=True, output_hidden_states = True, output_scores=True)
            output = model(prompt, output_hidden_states = True)#,return_dict_in_generate=True, output_scores=True)
        
            #output = output[1]
            hidden_states = output.hidden_states
            hidden_states = torch.stack(hidden_states, dim = 0).squeeze()
            hidden_states = hidden_states.detach().cpu().numpy()
            #head_wise_hidden_states = [ret[head].output.squeeze().detach().cpu() for head in HEADS]
            # change from ret[head].output  to ret[head].input

            ## get attention inputs before merging transformation layer ,
            ## https://github.com/likenneth/honest_llama/issues/7
            head_wise_hidden_states = [ret[head].input.squeeze().detach().cpu() for head in HEADS]
            head_wise_hidden_states = torch.stack(head_wise_hidden_states, dim = 0).squeeze().numpy()
            mlp_wise_hidden_states = [ret[mlp].output.squeeze().detach().cpu() for mlp in MLPS]
            mlp_wise_hidden_states = torch.stack(mlp_wise_hidden_states, dim = 0).squeeze().numpy()

    return hidden_states, head_wise_hidden_states, mlp_wise_hidden_states, output

In [4]:
prompt_reason = "What is the colour of the sky?"
inputs = tokenizer(prompt_reason, return_tensors="pt")
input_ids = inputs["input_ids"]#.to(device)
device = "cuda"
layer_wise_activations, head_wise_activations, _, output = get_llama_activations_bau(model, input_ids, device)
# get_llama_activations_bau(model, input_ids, "cuda")

In [7]:
num_layers = model.config.num_hidden_layers
num_heads = model.config.num_attention_heads
sequence_length = len(tokenizer(prompt_reason, return_tensors="pt")['input_ids'][0])
dimension_of_heads = model.config.hidden_size // num_heads
print(head_wise_activations.shape) ## [Layers, sequence_length, hidden_size]
head_wise_activations_reshaped = head_wise_activations.reshape(num_layers, sequence_length, num_heads, dimension_of_heads)
print(head_wise_activations_reshaped.shape) ## [Layers, sequence_length, number_of_heads, dimension_of_heads]

(32, 9, 4096)
(32, 9, 32, 128)


In [26]:
import numpy as np

num_heads = 32
head_dim = model.config.hidden_size // num_heads
attn_output = head_wise_activations
batch_size = 1
query_length = len(tokenizer(prompt_reason, return_tensors="pt")['input_ids'][0])

# Add a batch dimension and then reshape
attn_output = np.expand_dims(head_wise_activations, axis=0)  # Adds a batch dimension: shape becomes (1, 32, 9, 4096)
attn_output = attn_output.reshape(batch_size, query_length, num_heads, head_dim)  # Reshape to the desired shape

print(attn_output.shape)  # Should be (1, 9, 32, 128)

# Step 1: Reshape to separate the heads and head dimensions
#attn_output = attn_output.reshape(query_length, num_heads, head_dim)

ValueError: cannot reshape array of size 1179648 into shape (1,9,32,128)

In [27]:
expected_size = batch_size * query_length * num_heads * head_dim
expected_size

36864

In [20]:
attn_output.shape

(32, 9, 4096)

In [12]:
head_wise_activations[:,-1,:].shape

(32, 4096)

In [13]:
model.config

MistralConfig {
  "_name_or_path": "openchat/openchat_3.5",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.35.0",
  "use_cache": true,
  "vocab_si

In [15]:
tokenizer.vocab['<|pad_0|>']

32001

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM#, # AutoModelForCausalLM, AutoTokenizer#, LlamaForCausa
import torch 

MODEL_NAME ="openchat/openchat_3.5"#"likenneth/honest_llama2_chat_7B"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5"#"openchat/openchat_3.5"#"lmsys/vicuna-13b-v1.5" #"mistralai/Mistral-7B-v0.1"
LOAD_8BIT = True 
# #LlamaForCausalLM.from_pretrained(MODEL_NAME,
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
        load_in_8bit=LOAD_8BIT,
        torch_dtype=torch.float16,
        device_map="auto",
        )
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if "open" in model.config._name_or_path:

    model.generation_config.pad_token_id = 0

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [70]:
HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(model.config.num_hidden_layers)]
#HEADS = [f"model.layers.{i}.self_attn.o_proj" for i in range(model.config.num_hidden_layers)]
MLPS = [f"model.layers.{i}.mlp" for i in range(model.config.num_hidden_layers)]
#print(HEADS)
device = "cuda"
with torch.no_grad():
    
    prompt = prompt.to(device)
    with TraceDict(model, HEADS+MLPS) as ret:
        output = model(prompt, output_hidden_states = True)#,return_dict_in_generate=True, output_scores=True)
    
    #output = output[1]
    #hidden_states = output.hidden_states
    #hidden_states = torch.stack(hidden_states, dim = 0).squeeze()
    #hidden_states = hidden_states.detach().cpu().numpy()
    head_wise_hidden_states = [ret[head].output.squeeze().detach().cpu() for head in HEADS]
    head_wise_hidden_states = torch.stack(head_wise_hidden_states, dim = 0).squeeze().numpy()
    mlp_wise_hidden_states = [ret[mlp].output.squeeze().detach().cpu() for mlp in MLPS]
    mlp_wise_hidden_states = torch.stack(mlp_wise_hidden_states, dim = 0).squeeze().numpy()

In [88]:
len(HEADS)

32

In [8]:
#model.model#.layers[0]#.self_attn.o_proj

In [3]:
#head_wise_hidden_states#.shape()

In [80]:
#[head for head in HEADS]

In [85]:
ret[HEADS[0]].output.shape#.squeeze().detach().cpu().shape

torch.Size([1, 20, 4096])

In [72]:
head_wise_hidden_states.shape

(32, 20, 4096)

NameError: name 'head_wise_hidden_states' is not defined

In [76]:
mlp_wise_hidden_states = [ret[mlp].output.squeeze().detach().cpu() for mlp in MLPS]
len(mlp_wise_hidden_states)#.shape

32

In [77]:
mlp_wise_hidden_states[0].shape

torch.Size([20, 4096])

In [ ]:
mlp_wise_hidden_states[0].shape

In [ ]:

#prompt_reason = "What is the colour of the sky?"
#prompt_reason = ["What is the colour of the sky?", ]
tokenizer.pad_token_id = 32001#.special_token#.config
prompt_1 = "What is the colour of the sky?"
prompt_2 = "What is the colour of the sky?\n\nThe sky is blue.\n\nWhy is"

device = "cuda"
inputs = tokenizer(prompt_reason, return_tensors="pt", padding= True)
prompt = inputs["input_ids"].to(device)
with torch.no_grad():
    inputs = tokenizer(prompt_1, return_tensors="pt", padding= True)
    prompt = inputs["input_ids"].to(device)
    output_1 = model.generate(prompt,return_dict_in_generate=True, output_hidden_states = True, temperature = 0, output_scores= True)#, output_scores=True, )
    
    inputs = tokenizer(prompt_2, return_tensors="pt", padding= True)
    prompt = inputs["input_ids"].to(device)
    output_2 = model(prompt, output_hidden_states = True)#, output_scores=True 

#### Hidden_states tranformation between 1st and 2nd generation

In [66]:

#prompt_reason = "What is the colour of the sky?"
#prompt_reason = ["What is the colour of the sky?", ]
tokenizer.pad_token_id = 32001#.special_token#.config
prompt_1 = "What is the colour of the sky?"
prompt_2 = "What is the colour of the sky?\n\nThe sky is blue.\n\nWhy is"

device = "cuda"
inputs = tokenizer(prompt_reason, return_tensors="pt", padding= True)
prompt = inputs["input_ids"].to(device)
with torch.no_grad():
    inputs = tokenizer(prompt_1, return_tensors="pt", padding= True)
    prompt = inputs["input_ids"].to(device)
    output_1 = model.generate(prompt,return_dict_in_generate=True, output_hidden_states = True, temperature = 0, output_scores= True)#, output_scores=True, )
    
    inputs = tokenizer(prompt_2, return_tensors="pt", padding= True)
    prompt = inputs["input_ids"].to(device)
    output_2 = model(prompt, output_hidden_states = True)#, output_scores=True

print("Output_1")
prompt_length = len(tokenizer(prompt_1, return_tensors="pt", padding= True)['input_ids'][0])
print("Length prompt",prompt_length)
print("Number of generated tokens:")
print(len(output_1.hidden_states))
print(len(output_1.scores))
## first token both hidden states 
layer = 0 
token = 3
## hidden_state = [num_tokens,layer, batch_size, number_of_processed_tokens, dimension]
number_of_processed_tokens = 0
sequence = 0
print("Number of Layers:")
print(len(output_1.hidden_states[token]))
print(f"Dimension of hidden state of token: {token} and layer: {layer}")
print(output_1.hidden_states[token][layer].shape)
print(f"Hidden state of token: {token} and layer: {layer} of batch number: 0")
print(output_1.hidden_states[token][layer][sequence].shape)
print(f"Hidden state of token: {token} and layer: {layer} of batch number: {sequence} and processed token: {token}")
print(output_1.hidden_states[token][layer][sequence][number_of_processed_tokens])

print("_________________")
print("Output_2")
number_of_processed_tokens = prompt_length+token-1 if token != 0  else number_of_processed_tokens#number_of_processed_tokens + token
print("Current index for hidden_state: ", number_of_processed_tokens)
print("Number of Layers:")
print(len(output_2.hidden_states))

print(f"Dimension of hidden state of layer: {layer}")
print(output_2.hidden_states[layer].shape)
print(f"Hidden state of token: {number_of_processed_tokens} and layer: {layer} of batch number: {sequence}")
print(output_2.hidden_states[layer][sequence].shape)#.shape)
print(output_2.hidden_states[layer][sequence][number_of_processed_tokens])#.shape)


#### Is the encoding by the model.generate as model()

Output_1
Length prompt 9
Number of generated tokens:
11
11
Number of Layers:
33
Dimension of hidden state of token: 3 and layer: 0
torch.Size([1, 1, 4096])
Hidden state of token: 3 and layer: 0 of batch number: 0
torch.Size([1, 4096])
Hidden state of token: 3 and layer: 0 of batch number: 0 and processed token: 3
tensor([ 0.0014, -0.0018, -0.0018,  ...,  0.0014,  0.0017, -0.0006],
       device='cuda:0', dtype=torch.float16)
_________________
Output_2
Current index for hidden_state:  11
Number of Layers:
33
Dimension of hidden state of layer: 0
torch.Size([1, 20, 4096])
Hidden state of token: 11 and layer: 0 of batch number: 0
torch.Size([20, 4096])
tensor([ 0.0014, -0.0018, -0.0018,  ...,  0.0014,  0.0017, -0.0006],
       device='cuda:0', dtype=torch.float16)


In [25]:
print(output_1.hidden_states[0][token][layer][0])

tensor([ 1.2684e-04,  2.9945e-04, -4.6730e-05,  ..., -5.3644e-07,
         1.7643e-04, -1.3173e-05], device='cuda:0', dtype=torch.float16)


In [122]:
print(len(output_1.hidden_states))#[0][token]

11


tensor([-0.0045,  0.0003, -0.0052,  ..., -0.0004, -0.0012,  0.0002],
       device='cuda:0', dtype=torch.float16)


tensor([-0.0045,  0.0003, -0.0052,  ..., -0.0004, -0.0012,  0.0002],
       device='cuda:0', dtype=torch.float16)

In [51]:
#output.hidden_states

In [80]:

prompt_reason = "What is the colour of the sky?"

inputs = tokenizer(prompt_reason, return_tensors="pt")
input_ids = inputs["input_ids"]#.to(device)
device = "cuda"
layer_wise_activations, head_wise_activations, _, output = get_llama_activations_bau(model, input_ids, device)
# get_llama_activations_bau(model, input_ids, "cuda")

In [84]:
len(output.hidden_states)

33

In [59]:
layer_wise_activations.shape

(33, 9, 4096)

In [90]:

print(input_ids.shape)
## Length number of generated tokens 
#print(len(output.scores))
## Length of hidden states 
print(len(output.hidden_states))#[-1][-1].shape
layer_wise_activations.shape
complete_output = output
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
#print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.hidden_states[0][-1].shape)#[-1][-1].shape

torch.Size([1, 9])
33
9
33
torch.Size([9, 4096])
torch.Size([9, 4096])


In [76]:

print(input_ids.shape)
## Length number of generated tokens 
print(len(output.scores))
## Length of hidden states 
print(len(output.hidden_states))#[-1][-1].shape
layer_wise_activations.shape
complete_output = output
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.hidden_states[0][-1].shape)#[-1][-1].shape

torch.Size([1, 9])
11
11
9
11
11
torch.Size([1, 1, 4096])
torch.Size([1, 9, 4096])


In [72]:
print(len(output.hidden_states[0][0]))

1


In [75]:
output.hidden_states[0][]].shape

torch.Size([1, 9, 4096])

In [68]:
len(output.scores)

11

In [28]:
#output.loss#.['scores']

In [80]:
# model.model.layers[0].self_attn.head_out   #.layers

In [9]:


print(len(attentions['layer_-1_head_0_last_token']))

465

In [14]:
print(len(attentions['layer_-16_head_30_last_token']))

465


In [30]:


#output, complete_output = evaluate(prompt_reason, model = model, tokenizer= tokenizer, stopping_criteria = None, device = "cuda", **extra_args) #
device = "cuda"
layer_wise_activations, head_wise_activations, _ = get_llama_activations_bau(model, prompt_reason, device)

AttributeError: 'str' object has no attribute 'to'

In [10]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.attentions))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.attentions[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.attentions[0][-1].shape)#[-1][-1].shape

352
114
114
torch.Size([1, 32, 1, 465])
torch.Size([1, 32, 352, 352])


In [17]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.attentions))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.attentions[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.attentions[0][-1].shape)#[-1][-1].shape

9
600
600
torch.Size([1, 32, 1, 608])
torch.Size([1, 32, 9, 9])


In [22]:
complete_output.attentions[-1][-1][0].shape

torch.Size([32, 1, 608])

In [27]:
complete_output.attentions[-1][-1].shape#[0][0][0]#.shape

torch.Size([1, 32, 1, 608])

In [ ]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.attentions))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.attentions[-1][-1].shape)#[-1][-1].shape
## Dimension of attentions of first generated token and last layer
print(complete_output.attentions[0][-1].shape)#[-1][-1].shape

In [18]:
print(complete_output.attentions[0][-1].shape)#[-1][-1].shape

torch.Size([1, 32, 9, 9])


In [22]:
complete_output.scores[-1],1

(tensor([[-7.2734, -6.3906,  9.2891,  ..., -2.6016, 13.9531, -2.3145]],
        device='cuda:0'),
 1)

In [9]:
len(complete_output.sequences[0])#.shape

427

In [23]:
tokenizer.decode(complete_output.sequences[0])

'<s> GPT4 Correct User: System: |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|\n|JWST| |orbits| |Second LaGrange Point (L2)|\n|JWST| |maintains_ZExcursion| "≤500,000 Km"\n|JWST| |maintains_YExcursion| "≤800,000 Km"\n|JWST| |avoids| |Eclipse|\n|JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|\n|JWST| |has_MissionLifetime| "5 years"\n|JWST| |completes_Commissioning| within "6 months"\n|JWST System| |delivers_RealTimeTelemetry| "92.5%"\n|JWST System| |archives_RecordedData| "97%"\n|JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|\nRequirement: The planned commissioning phase shall end no later than six months after launch.\nBased on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No". Use Python scripting if you need to calculate something by starting with \'\'\'python... Let\'s think step-by-step. \n\n1. The commissioning phase is mentioned as "completes_Commissioning withi

In [17]:
print(output)

<s> GPT4 Correct User: System: |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|
|JWST| |orbits| |Second LaGrange Point (L2)|
|JWST| |maintains_ZExcursion| "≤500,000 Km"
|JWST| |maintains_YExcursion| "≤800,000 Km"
|JWST| |avoids| |Eclipse|
|JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|
|JWST| |has_MissionLifetime| "5 years"
|JWST| |completes_Commissioning| within "6 months"
|JWST System| |delivers_RealTimeTelemetry| "92.5%"
|JWST System| |archives_RecordedData| "97%"
|JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|
Requirement: The planned commissioning phase shall end no later than six months after launch.
Based on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No". Use Python scripting if you need to calculate something by starting with '''python... Let's think step-by-step. 

1. The commissioning phase is mentioned as "completes_Commissioning within "6 months"". 
2. 

In [ ]:
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

In [6]:
len(complete_output.attentions)

114

In [15]:
complete_output.scores[0]#)#.shape

tensor([[-7.3008, -7.4531,  1.6777,  ..., -3.0215,  5.3516, -2.2734]],
       device='cuda:0')

In [16]:
complete_output.scores[0].shape#)#.shape

torch.Size([1, 32002])

In [19]:
import torch 

torch.exp(torch.max(complete_output.scores[0]))# .shape

tensor(4.3635e-09, device='cuda:0')

In [6]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.attentions))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.attentions[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.attentions[0][-1].shape)#[-1][-1].shape

352
114
114
torch.Size([1, 32, 1, 465])
torch.Size([1, 32, 352, 352])


In [9]:
layer = -8
head = 8

## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.attentions))#[-1][-1].shape
## Dimension of hidden state of last generated token and [layer] layer
print(complete_output.attentions[-1][layer][head].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and [layer] layer
print(complete_output.attentions[0][layer].shape)#[-1][-1].shape

352
114
114
torch.Size([1, 32, 1, 465])
torch.Size([1, 32, 352, 352])


In [16]:
layer = -8
head = 8
sequence = -1
complete_output.attentions[-1][layer][sequence][head][-1].shape


torch.Size([465])

In [19]:
#[num for num in range(32)]

In [17]:
complete_output

NameError: name 'compelte_output' is not defined

In [15]:
465*32/8

1860.0

In [8]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.hidden_states[0][-1].shape)#[-1][-1].shape

352
114
114
torch.Size([1, 1, 4096])
torch.Size([1, 352, 4096])


In [8]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.hidden_states[0][-1].shape)#[-1][-1].shape

352
114
114
torch.Size([1, 1, 4096])
torch.Size([1, 352, 4096])


In [28]:
layer = -1
complete_output.hidden_states[-1][layer][-1][-1].shape

torch.Size([4096])

In [31]:
complete_output.hidden_states[0][layer][-1][-1]#.shape

tensor([-4.0195,  2.8145, -0.4246,  ..., -2.5488,  6.7031,  1.4111],
       device='cuda:0', dtype=torch.float16)

In [36]:
complete_output.hidden_states[0][layer][0][-1].shape

torch.Size([4096])

In [33]:
complete_output.hidden_states[0][layer][-1][-1]#.shape#[-1]#.shape

tensor([-4.0195,  2.8145, -0.4246,  ..., -2.5488,  6.7031,  1.4111],
       device='cuda:0', dtype=torch.float16)

In [30]:
complete_output.hidden_states[8][layer][-1][-1].shape

torch.Size([4096])

In [11]:
#tokenizer.decode(complete_output.sequences[0][8])
complete_output.hidden_states[-1][-1][-1].shape

torch.Size([1, 4096])

In [18]:
#tokenizer.decode(complete_output.sequences[0][8])
complete_output.hidden_states[0][-1][-1][-1:]#.shape

tensor([[-4.0195,  2.8145, -0.4246,  ..., -2.5488,  6.7031,  1.4111]],
       device='cuda:0', dtype=torch.float16)

In [19]:
complete_output.hidden_states[0][-1][-1][-1]#

tensor([-4.0195,  2.8145, -0.4246,  ..., -2.5488,  6.7031,  1.4111],
       device='cuda:0', dtype=torch.float16)

In [9]:
print(complete_output.hidden_states[0][8][-1][-1:].shape)#[-1].shape)

torch.Size([1, 4096])


In [18]:
print(complete_output.hidden_states[0][-1][-1][-1:].shape)#[-1].shape)

torch.Size([1, 4096])


In [20]:
complete_output.hidden_states[0][-1][-1][-1:]

tensor([[-4.0195,  2.8145, -0.4246,  ..., -2.5488,  6.7031,  1.4111]],
       device='cuda:0', dtype=torch.float16)

In [21]:
complete_output.hidden_states[0][-1][-1][-1]

tensor([-4.0195,  2.8145, -0.4246,  ..., -2.5488,  6.7031,  1.4111],
       device='cuda:0', dtype=torch.float16)

In [8]:
len(complete_output.hidden_states)#[-1].shape

114

In [24]:
output

'<s> GPT4 Correct User: System: |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|\n|JWST| |orbits| |Second LaGrange Point (L2)|\n|JWST| |maintains_ZExcursion| "≤500,000 Km"\n|JWST| |maintains_YExcursion| "≤800,000 Km"\n|JWST| |avoids| |Eclipse|\n|JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|\n|JWST| |has_MissionLifetime| "5 years"\n|JWST| |completes_Commissioning| within "6 months"\n|JWST System| |delivers_RealTimeTelemetry| "92.5%"\n|JWST System| |archives_RecordedData| "97%"\n|JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|\nRequirement: The planned commissioning phase shall end no later than six months after launch.\nBased on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No". Use Python scripting if you need to calculate something by starting with \'\'\'python... Let\'s think step-by-step. \n\n1. The commissioning phase of the JWST system is mentioned to be complete

In [23]:
print(complete_output.hidden_states[-1][-1][-1].shape)#[-1][-1].shape

torch.Size([1, 4096])


In [67]:
print(output)

<s> Please answer the following question. Are there 50 states in the US. Answer (Yes or No ONLY): Yes


In [33]:
len(tokenizer.encode(prompt_reason))

896

In [50]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

896
1
1
torch.Size([1, 896, 4096])


In [55]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

9
600
600
torch.Size([1, 1, 4096])


In [11]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.hidden_states[0][-1].shape)#[-1][-1].shape

9
600
600
torch.Size([1, 1, 4096])
torch.Size([1, 9, 4096])


In [4]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape
## Dimension of hidden state of first generated token and last layer
print(complete_output.hidden_states[0][-1].shape)#[-1][-1].shape

9
600
600
torch.Size([1, 1, 4096])
torch.Size([1, 9, 4096])


In [68]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

25
1
1
torch.Size([1, 25, 4096])


In [58]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

18
13
13
torch.Size([1, 1, 4096])


In [46]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

856
134
134
torch.Size([1, 1, 4096])


In [53]:
## Length tokenized input
print(len(tokenizer.encode(prompt_reason)))
## Length number of generated tokens 
print(len(complete_output.scores))
## Length of hidden states 
print(len(complete_output.hidden_states))#[-1][-1].shape
## Dimension of hidden state of last generated token and last layer
print(complete_output.hidden_states[-1][-1].shape)#[-1][-1].shape

856
73
73
torch.Size([1, 1, 4096])


In [7]:
## hidden state of last layer ([-1]) is a tensor of shape (batch_size, sequence_length, hidden_size)
print(complete_output.hidden_states[0][-1].shape)#[].shape#
## get hidden state of "yes" token in last layer ([-1]) 
print(complete_output.hidden_states[0][-1][0][-1].shape)#[].shape#
#https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/output#transformers.utils.ModelOutput
#https://github.com/huggingface/transformers/issues/26174
import torch 
import json 

layers = [-1, -8, -16, -24]

output_hidden_states = {}
for layer in layers:

    ## get hidden state of last token in specific layer ([-1]) 
    tensor = complete_output.hidden_states[-1][layer][0][-1]
    print(complete_output.hidden_states[-1][layer][0][-1])
    # Move to CPU and convert to float32 or float64
    tensor =  tensor.to(device='cpu', dtype=torch.float32)
    # Convert to a list
    tensor_list = tensor.tolist()
    # Serialize to JSON
    #json_output = json.dumps(tensor_list)

    output_hidden_states[layer] = tensor_list

#output_hidden_states

torch.Size([1, 9, 4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])


In [9]:
## hidden state of last layer ([-1]) is a tensor of shape (batch_size, sequence_length, hidden_size)
print(complete_output.hidden_states[0][-1].shape)#[].shape#
## get hidden state of "yes" token in last layer ([-1]) 
print(complete_output.hidden_states[0][-1][0][-1].shape)#[].shape#
#https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/output#transformers.utils.ModelOutput
#https://github.com/huggingface/transformers/issues/26174
import torch 
import json 

layers = [-1, -8, -16, -24]

output_hidden_states = {}
for layer in layers:

    ## get hidden state of last token in specific layer ([-1]) 
    tensor = complete_output.hidden_states[0][layer][0][-1]
    # Move to CPU and convert to float32 or float64
    tensor =  tensor.to(device='cpu', dtype=torch.float32)
    # Convert to a list
    tensor_list = tensor.tolist()
    # Serialize to JSON
    #json_output = json.dumps(tensor_list)

    output_hidden_states[layer] = tensor_list

#output_hidden_states

torch.Size([1, 856, 4096])
torch.Size([4096])


In [69]:
#model.model.layers[0]#[-1]#[-1]#ModuleList#[-1]#.layers

In [35]:
import torch 
import json
# Optionally, save to a file
with open('tensor_output_test.json', 'w') as f:
    f.write(json_output)

In [7]:
#print(output[0])

print(torch.argmax(complete_output.scores[0]))
print(tokenizer.decode(torch.argmax(complete_output.scores[0])))
scores = torch.softmax(complete_output.scores[-1],1)
print(torch.max(scores).item())

tensor(5613, device='cuda:0')
Yes
0.4222647249698639


In [93]:
## inverse tokenizer dict
inv_vocab = {v:k for k, v in tokenizer.vocab.items()}
print(inv_vocab[5613])
print(inv_vocab[28802])
print(inv_vocab[5592])
print(inv_vocab[13])
print(inv_vocab[2501])


Yes
Y
▁Yes
<0x0A>
No


In [35]:
from test_reasoning import evaluate, generate_cpu_gpu
from transformers import GenerationConfig
import os 

MAX_NEW_TOKENS = 512
extra_args = {  "max_new_tokens":MAX_NEW_TOKENS,
                "min_new_tokens": 256, 
                "do_sample":True, 
                "num_beams" : 1, 
                "num_return_sequences" : 1, 
                "temperature": 0.8, 
                "top_p": 0.95,
                "begin_suppress_tokens": [2], 
                }

req= "The Observatory shall orbit the Second LaGrange Point (L2) of the Sun–Earth system."
req = "The operational JWST System shall maintain the excursion of the orbit about L2 in the Z direction (defined in Figure 3-2) to less than or equal to 500,000 Km."
definition = """|JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|
    |JWST| |orbits| |Second LaGrange Point (L2)|
    |JWST| |maintains_ZExcursion| "≤500,000 Km"|
    |JWST| |maintains_YExcursion| "≤800,000 Km"|
    |JWST| |avoids| |Eclipse|
    |JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|
    |JWST| |has_MissionLifetime| "5 years"|
    |JWST| |completes_Commissioning| within "6 months"|
    |JWST System| |delivers_RealTimeTelemetry| "92.5%"|
    |JWST System| |archives_RecordedData| "97%"|
    |JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|
    """ 

prompt_reason = """Based on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No" . Use Python scripting if you need to calculate something by starting with '''python... Let's think step-by-step: """
system_prompt = "ASSISTANT:"#"GPT4 Correct Assistant: "
user_prompt = "HUMAN:"#"GPT4 Correct User:"

prompt_reason = f"{user_prompt}System: {definition}\nRequirement: {req}\n{prompt_reason}\n{system_prompt}" 

script_dir = ""

prompt_path = os.path.join(script_dir, './Machine_Learning/prompt_reason.txt')
with open(prompt_path, 'r') as f:
        prompt_reason = f.read()

prompt_reason = f"{prompt_reason}\n System:{definition} \nRequirement: {req} \nASSISTANT: Let's analyze the given System and requirement step-by-step:"

print(prompt_reason)             
#generate_cpu_gpu(inputs, script_dir, generation_config)
output = evaluate(prompt_reason, model = model, tokenizer= tokenizer, device = "cuda", **extra_args)

HUMAN: I want you to act as an expert Spacecraft Engineer. I will provide an extract of an Engineering System in triple format as well as a requirement, and it will be your job to analyze if the requirement is satisfied by the engineering system or not. 
First try to extract the information (entities and relations) from the System that is potentially useful in your analysis. 
Next try to establish if that information either implicitly or explicitly fulfills the requirement. 
This could involve performing a gap analysis between the engineering system and the requirement. 
The Final step is to clearly state if the requirement is satisfied or not by stating FINAL ANSWER: YES or FINAL ANSWER: NO.
My first engineering system and requirement are: 

System = "|Data-Handling Subsystem| |directlyContainsEntity| |Service Module|
|Payload Module| |directlyContainsEntity| |Spacecraft|
|Payload Module| |directlyContainsEntity| |Brackets_Misc_PLM|
|Payload Module| |directlyContainsEntity| |Instrumen

### Randomising

In [ ]:
import random
# Set the random seed
random.seed(22)  # Replace 42 with your desired seed
# Make a copy of the list
rand_results = results.copy()
# Shuffle the copy
random.shuffle(rand_results)

definition = ""
for triples in rand_results: 

    definition += "|" + triples[3] +"|" +" " + "|" + triples[4] +"|" +" " +"|"+ triples[5] +"|" + "\n"

### Parsing

In [1]:
output = """<s> GPT4 Correct User:System: |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|
    |JWST| |orbits| |Second LaGrange Point (L2)|
    |JWST| |maintains_ZExcursion| "≤500,000 Km"|
    |JWST| |maintains_YExcursion| "≤800,000 Km"|
    |JWST| |avoids| |Eclipse|
    |JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|
    |JWST| |has_MissionLifetime| "5 years"|
    |JWST| |completes_Commissioning| within "6 months"|
    |JWST System| |delivers_RealTimeTelemetry| "92.5%"|
    |JWST System| |archives_RecordedData| "97%"|
    |JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|
    
Requirement: The Launch Vehicle shall place the Observatory on a trajectory from which the Observatory can transfer itself to its operational orbit.
Based on the system description, is this requirement satisfied? Include in your answer "Final Answer: Yes/No". Use Python scripting if you need to calculate something by starting with '''python... 
GPT4 Correct Assistant:Let's analyze the given System and requirement step-by-step:

System:
- |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|

Requirement:
- The Launch Vehicle shall place the Observatory on a trajectory from which the Observatory can transfer itself to its operational orbit.

From the given system information, we can see that the Launch Vehicle is responsible for placing the JWST in a Transfer Orbit. This transfer orbit is the trajectory from which the JWST can transfer itself to its operational orbit.

Final Answer: Yes.<|end_of_turn|>""" 

def parse_output(output, system_prompt, cot=True):

    if cot: 
        output = output.split("Let's analyze the given System and requirement step-by-step:")[-1]        
    else: 
        output = output.split(system_prompt)[-1]

    return output
    
print(parse_output(output, "GPT4 Correct Assistant:", cot= True))



System:
- |JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|

Requirement:
- The Launch Vehicle shall place the Observatory on a trajectory from which the Observatory can transfer itself to its operational orbit.

From the given system information, we can see that the Launch Vehicle is responsible for placing the JWST in a Transfer Orbit. This transfer orbit is the trajectory from which the JWST can transfer itself to its operational orbit.

Final Answer: Yes.<|end_of_turn|>


### Testing evaluate function

In [ ]:
# from test_reasoning import evaluate, generate_cpu_gpu
# from transformers import GenerationConfig


# MAX_NEW_TOKENS = 512
# extra_args = {"do_sample":True, "max_new_tokens": MAX_NEW_TOKENS, "num_return_sequences" : 1}
# temperature=0.8
# #top_k = generation_config.top_k, 
# num_beams= 1
# top_p = 0.95
# num_return_sequences=1
# no_repeat_ngram_size = 8


# generation_config = GeneraAtionConfig(
#         temperature=temperature,
#         top_p=top_p,
#         #top_k=top_k,
#         num_beams=num_beams,
#         no_repeat_ngram_size= no_repeat_ngram_size,
#         **extra_args,
#      )

# prompt_reason = test 

# output = evaluate(prompt_reason, generation_config, model = model, tokenizer= tokenizer, device = "cuda")[0]

In [ ]:
MAX_NEW_TOKEN = 512
extra_args = {  "max_new_tokens":MAX_NEW_TOKEN,
                "min_new_tokens": 256, 
                "do_sample":True, 
                "num_beams" : 1, 
                "num_return_sequences" : 1, 
                #"no_repeat_ngram_size": 12, 
                "temperature": 0.8, 
                "top_p": 0.95,
                "begin_suppress_tokens": [2], 
                }

inputs = tokenizer(prompt_reason, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)
generation_config = GenerationConfig(**extra_args)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
    )
    s = generation_output.sequences[0]
output = tokenizer.decode(s)

In [ ]:
from test_reasoning import evaluate, generate_cpu_gpu

from transformers import GenerationConfig
import os 
generation_config = GenerationConfig()

prompt_path = os.path.join("", './Machine_Learning/prompt_reason.txt')
with open(prompt_path, 'r') as f:
        prompt_reason = f.read()

req= "The Observatory shall orbit the Second LaGrange Point (L2) of the Sun–Earth system."
req = "The operational JWST System shall maintain the excursion of the orbit about L2 in the Z direction (defined in Figure 3-2) to less than or equal to 500,000 Km."

definition = """|JWST| |isPlacedIn| |Transfer Orbit| by |Launch Vehicle|
    |JWST| |orbits| |Second LaGrange Point (L2)|
    |JWST| |maintains_ZExcursion| "≤500,000 Km"|
    |JWST| |maintains_YExcursion| "≤800,000 Km"|
    |JWST| |avoids| |Eclipse|
    |JWST| |transfersTo| |L2 Operational Orbit| using |DUA IRD (JWST-IRD-003674)|
    |JWST| |has_MissionLifetime| "5 years"|
    |JWST| |completes_Commissioning| within "6 months"|
    |JWST System| |delivers_RealTimeTelemetry| "92.5%"|
    |JWST System| |archives_RecordedData| "97%"|
    |JWST| |achieves_Sensitivity| per |NIRCam, NIRSpec, MIRI, FGS-TF Calculations|
    """ 

MAX_NEW_TOKEN = 512

prompt_reason = f"{prompt_reason}\n System:{definition} \nRequirement: {req} \nASSISTANT: Let's analyze the given System and requirement step-by-step:"

output, complete_output = evaluate(prompt_reason,model = model, tokenizer = tokenizer, device= "cuda", temperature=0.8, top_k = generation_config.top_k, num_beams= 1, top_p = 0.95, do_sample=True, max_new_tokens=MAX_NEW_TOKEN, num_return_sequences=1)

In [ ]:
temperature=0.8
top_k = generation_config.top_k 
num_beams= 1
top_p = 0.95
no_repeat_ngram_size = 8
device = 'cuda'


MAX_NEW_TOKEN = 512

keyargs = {"do_sample":True, "num_return_sequences": 1}
max_new_tokens = MAX_NEW_TOKEN

inputs = tokenizer(prompt_reason, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)
generation_config = GenerationConfig(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    num_beams=num_beams,
    no_repeat_ngram_siz= no_repeat_ngram_size,
    **keyargs,
)

#print(temperature)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
    )
    s = generation_output.sequences[0]
output = tokenizer.decode(s)